# CV Project 3 - Image Colourization on pets dataset

#### *Project description* - Colourization of pets images with use of the neural network.

Prepared by:
 - Marianna Myszkowska 156041
 - Jakub Liszyński 156060